In [1]:
!pip install mlflow
import mlflow
mlflow.__version__

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.2 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.2 MB 368.7 kB/s eta 0:00:27
   -- ------------------------------------- 0.5/10.2 MB 368.7 kB/s eta 0:00:27
   -- ------------------------------------- 0.5/10.2 MB 368.7 kB/s eta 0:00:27
   -- ------------------------------------- 0.5/10.2 MB 368.7 kB/s eta 0:00:27
   -- ------------------------------------- 0.5/10.2 MB 368.7 kB/s eta 0:00:27
   --- ------------------------------


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\ingin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
C:\Users\ingin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'3.10.0'

In [4]:
mlflow.set_tracking_uri("http://32.192.36.226:5000")

experiment_name = "mi-primer-experimento"
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

if experiment is not None and experiment.lifecycle_stage == "deleted":
    # Restore the deleted experiment
    client.restore_experiment(experiment.experiment_id)

mlflow.set_experiment(experiment_name)

with mlflow.start_run():
    mlflow.log_param("learning_rate", 0.01)
    mlflow.log_param("epochs", 10)
    mlflow.log_metric("accuracy", 0.92)
    mlflow.log_metric("loss", 0.08)
    print("Experimento registrado!")

Experimento registrado!
🏃 View run agreeable-kite-559 at: http://32.192.36.226:5000/#/experiments/1/runs/87c3cca941d84a6cbd6c7cc35fb775ee
🧪 View experiment at: http://32.192.36.226:5000/#/experiments/1


Instalar dependencias

In [5]:
!pip install mlflow scikit-learn pandas numpy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\ingin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
# Apuntar a tu servidor MLflow en EC2
mlflow.set_tracking_uri("http://32.192.36.226:5000")
mlflow.set_experiment("german-credit-risk")

print(f"Tracking URI: {mlflow.get_tracking_uri()}")

Tracking URI: http://32.192.36.226:5000


In [9]:
import pandas as pd

URL = "https://raw.githubusercontent.com/camilousa/datasets/refs/heads/master/german_credit_data(2)(1).csv"
df = pd.read_csv(URL, index_col=0)

print(f"Shape: {df.shape}")
print(df.head())
print(df.dtypes)
print(df['Risk'].value_counts())

Shape: (1000, 10)
    Age     Sex  Job Housing Saving_accounts Checking_account  Credit_amount  \
id                                                                             
0    67    male    2     own             NaN           little           1169   
1    22  female    2     own          little         moderate           5951   
2    49    male    1     own          little              NaN           2096   
3    45    male    2    free          little           little           7882   
4    53    male    2    free          little           little           4870   

    Duration              Purpose  Risk  
id                                       
0          6             radio/TV  good  
1         48             radio/TV   bad  
2         12            education  good  
3         42  furniture/equipment  good  
4         24                  car   bad  
Age                  int64
Sex                 object
Job                  int64
Housing             object
Saving_accounts    

In [11]:
from sklearn.preprocessing import LabelEncoder

df_clean = df.copy()
df_clean['Saving_accounts'] = df_clean['Saving_accounts'].fillna('unknown')
df_clean['Checking_account'] = df_clean['Checking_account'].fillna('unknown')

# Encodear variables categóricas
le = LabelEncoder()
cat_cols = ['Sex', 'Housing', 'Saving_accounts', 'Checking_account', 'Purpose']
for col in cat_cols:
    df_clean[col] = le.fit_transform(df_clean[col])

# Separar features y target
X = df_clean.drop(columns=['Risk'])
y = le.fit_transform(df_clean['Risk'])  # good=1, bad=0

print(f"\nX shape: {X.shape}")
print(f"Distribución target — 0(bad): {sum(y==0)}, 1(good): {sum(y==1)}")


X shape: (1000, 9)
Distribución target — 0(bad): 300, 1(good): 700


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [15]:
from sklearn.ensemble import RandomForestClassifier

with mlflow.start_run(run_name="RandomForest_TrainTest"):

    # Parámetros del modelo
    n_estimators = 100
    max_depth = 5
    random_state = 42

    mlflow.log_param("model", "RandomForestClassifier")
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", random_state)
    mlflow.log_param("dataset", "german_credit_risk")
    mlflow.log_param("n_samples", len(df_clean))
    mlflow.log_param("n_features", X.shape[1])

    # Entrenar modelo
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state
    )
    model.fit(X_train, y_train)

🏃 View run RandomForest_TrainTest at: http://32.192.36.226:5000/#/experiments/2/runs/cee87f350b3c47da984f4dd6b8d79611
🧪 View experiment at: http://32.192.36.226:5000/#/experiments/2


In [18]:
# Métricas
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
train_accuracy = accuracy_score(y_train, model.predict(X_train))

mlflow.log_metric("train_accuracy_score", train_accuracy)
mlflow.log_metric("test_accuracy_score", test_accuracy)

# Guardar el modelo
mlflow.sklearn.log_model(model, "random_forest_model")
print(f"Train accuracy: {train_accuracy:.4f}")
print(f"Test accuracy:  {test_accuracy:.4f}")
print(f"Run ID: {mlflow.active_run().info.run_id}")

2026/02/26 21:32:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/26 21:32:38 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


Train accuracy: 0.7975
Test accuracy:  0.7600
Run ID: c98b88a3239e4d6691dc4fc8c944b383


Cross Validation

In [21]:
from sklearn.model_selection import cross_validate

mlflow.end_run()  # End any previous run if still active

with mlflow.start_run(run_name="RandomForest_CrossValidation"):

    n_estimators = 100
    max_depth = 5
    cv_folds = 10

    mlflow.log_param("model", "RandomForestClassifier")
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("cv", cv_folds)
    mlflow.log_param("dataset", "german_credit_risk")
    mlflow.log_param("n_samples", len(df_clean))

    model_cv = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    results = cross_validate(
        model_cv, X, y,
        cv=cv_folds,
        scoring='accuracy',
        return_train_score=True
    )

    train_accuracy = results["train_score"].mean()
    test_accuracy  = results["test_score"].mean()

    mlflow.log_metric("mean_train_accuracy_score", train_accuracy)
    mlflow.log_metric("mean_val_accuracy_score", test_accuracy)
    mlflow.log_metric("std_val_accuracy_score", results["test_score"].std())

    print(f"Mean train accuracy: {train_accuracy:.4f}")
    print(f"Mean val accuracy:   {test_accuracy:.4f}")
    print(f"Std val accuracy:    {results['test_score'].std():.4f}")
    print(f"Run ID: {mlflow.active_run().info.run_id}")

Mean train accuracy: 0.7876
Mean val accuracy:   0.7320
Std val accuracy:    0.0303
Run ID: 6df52ab1f13143188ada4306a55219f8
🏃 View run RandomForest_CrossValidation at: http://32.192.36.226:5000/#/experiments/2/runs/6df52ab1f13143188ada4306a55219f8
🧪 View experiment at: http://32.192.36.226:5000/#/experiments/2


XGBoost

In [22]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\ingin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, 
    recall_score, f1_score, roc_auc_score
)

In [24]:
def log_metrics_full(model, X_train, X_test, y_train, y_test, cv_folds=10):
    """Calcula y retorna todas las métricas de train/test y cross-validation"""
    
    # Train/Test métricas
    y_pred       = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    y_prob       = model.predict_proba(X_test)[:, 1]

    metrics = {
        "train_accuracy":  accuracy_score(y_train, y_pred_train),
        "test_accuracy":   accuracy_score(y_test, y_pred),
        "test_precision":  precision_score(y_test, y_pred, zero_division=0),
        "test_recall":     recall_score(y_test, y_pred, zero_division=0),
        "test_f1":         f1_score(y_test, y_pred, zero_division=0),
        "test_roc_auc":    roc_auc_score(y_test, y_prob),
    }

    # Cross-validation
    cv_results = cross_validate(
        model, X, y,
        cv=cv_folds,
        scoring='accuracy',
        return_train_score=True
    )
    metrics["cv_mean_train_accuracy"] = cv_results["train_score"].mean()
    metrics["cv_mean_val_accuracy"]   = cv_results["test_score"].mean()
    metrics["cv_std_val_accuracy"]    = cv_results["test_score"].std()

    return metrics

In [25]:
with mlflow.start_run(run_name="LogisticRegression_TrainTest"):

    # Parámetros
    C            = 1.0
    max_iter     = 1000
    cv_folds     = 10

    mlflow.log_param("model",        "LogisticRegression")
    mlflow.log_param("C",            C)
    mlflow.log_param("max_iter",     max_iter)
    mlflow.log_param("cv",           cv_folds)
    mlflow.log_param("scaler",       "StandardScaler")
    mlflow.log_param("dataset",      "german_credit_risk")
    mlflow.log_param("n_samples",    len(df_clean))
    mlflow.log_param("n_features",   X.shape[1])
    mlflow.log_param("test_size",    0.2)
    mlflow.log_param("random_state", 42)

    # Pipeline con escalado (necesario para Logistic Regression)
    model_lr = Pipeline([
        ("scaler", StandardScaler()),
        ("clf",    LogisticRegression(C=C, max_iter=max_iter, random_state=42))
    ])
    model_lr.fit(X_train, y_train)

    # Calcular y registrar métricas
    metrics = log_metrics_full(model_lr, X_train, X_test, y_train, y_test, cv_folds)
    for name, value in metrics.items():
        mlflow.log_metric(name, value)

    # Guardar modelo
    mlflow.sklearn.log_model(model_lr, "logistic_regression_model")

    print("=== Logistic Regression ===")
    for name, value in metrics.items():
        print(f"  {name}: {value:.4f}")
    print(f"  Run ID: {mlflow.active_run().info.run_id}")

2026/02/26 21:37:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/26 21:37:51 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


=== Logistic Regression ===
  train_accuracy: 0.7375
  test_accuracy: 0.7450
  test_precision: 0.7730
  test_recall: 0.9000
  test_f1: 0.8317
  test_roc_auc: 0.7636
  cv_mean_train_accuracy: 0.7417
  cv_mean_val_accuracy: 0.7310
  cv_std_val_accuracy: 0.0423
  Run ID: 46582d93fae442ffba5f3a6cd09d83af
🏃 View run LogisticRegression_TrainTest at: http://32.192.36.226:5000/#/experiments/2/runs/46582d93fae442ffba5f3a6cd09d83af
🧪 View experiment at: http://32.192.36.226:5000/#/experiments/2


In [26]:
with mlflow.start_run(run_name="XGBoost_TrainTest"):

    # Parámetros
    n_estimators     = 100
    max_depth        = 4
    learning_rate    = 0.1
    subsample        = 0.8
    cv_folds         = 10

    mlflow.log_param("model",          "XGBClassifier")
    mlflow.log_param("n_estimators",   n_estimators)
    mlflow.log_param("max_depth",      max_depth)
    mlflow.log_param("learning_rate",  learning_rate)
    mlflow.log_param("subsample",      subsample)
    mlflow.log_param("cv",             cv_folds)
    mlflow.log_param("dataset",        "german_credit_risk")
    mlflow.log_param("n_samples",      len(df_clean))
    mlflow.log_param("n_features",     X.shape[1])
    mlflow.log_param("test_size",      0.2)
    mlflow.log_param("random_state",   42)

    model_xgb = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        random_state=42,
        eval_metric="logloss",
        verbosity=0
    )
    model_xgb.fit(X_train, y_train)

    # Calcular y registrar métricas
    metrics = log_metrics_full(model_xgb, X_train, X_test, y_train, y_test, cv_folds)
    for name, value in metrics.items():
        mlflow.log_metric(name, value)

    # Guardar modelo
    mlflow.sklearn.log_model(model_xgb, "xgboost_model")

    print("=== XGBoost ===")
    for name, value in metrics.items():
        print(f"  {name}: {value:.4f}")
    print(f"  Run ID: {mlflow.active_run().info.run_id}")

2026/02/26 21:39:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/26 21:39:05 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


=== XGBoost ===
  train_accuracy: 0.9025
  test_accuracy: 0.7600
  test_precision: 0.7875
  test_recall: 0.9000
  test_f1: 0.8400
  test_roc_auc: 0.7727
  cv_mean_train_accuracy: 0.8968
  cv_mean_val_accuracy: 0.7580
  cv_std_val_accuracy: 0.0547
  Run ID: 90b2a567e3704c7196d1f38776fa4aaa
🏃 View run XGBoost_TrainTest at: http://32.192.36.226:5000/#/experiments/2/runs/90b2a567e3704c7196d1f38776fa4aaa
🧪 View experiment at: http://32.192.36.226:5000/#/experiments/2


In [27]:
import requests

PROD_URL = "http://52.91.237.16:8000"

# 1. Health check
print("=== Health Check ===")
print(requests.get(f"{PROD_URL}/health").json())

# 2. Predicción individual
print("\n=== Predicción Individual ===")
cliente = {
    "Age": 35,
    "Sex": 1,
    "Job": 2,
    "Housing": 1,
    "Saving_accounts": 1,
    "Checking_account": 1,
    "Credit_amount": 3000,
    "Duration": 24,
    "Purpose": 4
}
response = requests.post(f"{PROD_URL}/predict", json=cliente)
print(response.json())

# 3. Predicción batch (3 clientes)
print("\n=== Predicción Batch ===")
batch = [
    {"Age": 25, "Sex": 0, "Job": 2, "Housing": 2, "Saving_accounts": 0,
     "Checking_account": 1, "Credit_amount": 1500, "Duration": 12, "Purpose": 4},
    {"Age": 55, "Sex": 1, "Job": 3, "Housing": 1, "Saving_accounts": 4,
     "Checking_account": 3, "Credit_amount": 8000, "Duration": 36, "Purpose": 1},
    {"Age": 30, "Sex": 1, "Job": 1, "Housing": 0, "Saving_accounts": 0,
     "Checking_account": 0, "Credit_amount": 12000, "Duration": 60, "Purpose": 2},
]
response = requests.post(f"{PROD_URL}/predict/batch", json=batch)
print(response.json())

=== Health Check ===
{'status': 'healthy', 'model_loaded': True}

=== Predicción Individual ===
{'risk': 'good', 'probability_good': 0.842, 'probability_bad': 0.158, 'recommendation': 'Aprobar credito — bajo riesgo'}

=== Predicción Batch ===
{'total': 3, 'predicciones': [{'risk': 'good', 'probability_good': 0.6085, 'probability_bad': 0.3915, 'recommendation': 'Revisar manualmente — riesgo moderado'}, {'risk': 'good', 'probability_good': 0.9636, 'probability_bad': 0.0364, 'recommendation': 'Aprobar credito — bajo riesgo'}, {'risk': 'bad', 'probability_good': 0.1824, 'probability_bad': 0.8176, 'recommendation': 'Rechazar credito — alto riesgo'}]}
